In [ ]:
!pip install transformers datasets

In [ ]:
import torch
import pandas as pd
from numba import cuda
from datasets import load_dataset
# from GPUtil import showUtilization as gpu_usage
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, default_data_collator

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2',
                                        pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
print("The max model length is {} for this model".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))

The max model length is 1024 for this model
The beginning of sequence token <|endoftext|> token has the id 50256
The end of sequence token <|endoftext|> has the id 50256


In [ ]:
tokenizer.vocab_size

50257

In [ ]:
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [ ]:
tokenizer.all_special_tokens

['<|endoftext|>']

In [ ]:
tokenizer.eos_token_id

50256

In [ ]:
sentence = 'I am an Artificial Intelligence Developer'
input_ids  = tokenizer.encode(sentence, return_tensors = 'pt')

In [ ]:
input_ids

tensor([[   40,   716,   281, 35941,  9345, 23836]])

In [ ]:
tokenizer.decode(input_ids[0][3])

' Artificial'

In [ ]:
greedy_output = model.generate(input_ids, max_length=100, no_repeat_ngram_size=2)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
for i, output in enumerate(greedy_output):
    print("{}: {}...".format(i, tokenizer.decode(output,
                                                 skip_special_tokens=True)))
    print('')

0: I am an Artificial Intelligence Developer. I am a software developer. And I'm a programmer.

I'm not a computer scientist. But I do have a lot of experience in the field of Artificial intelligence. So I think that I can help you understand the challenges of AI. You can learn about the problems of artificial intelligence and how they can be solved. It's not just about solving problems. There are many other things that can go wrong. We can't just solve problems by solving them...



In [ ]:
beam_output = model.generate(input_ids,
                             max_length = 100,
                             num_beams=5,
                             num_return_sequences=5,
                             no_repeat_ngram_size=2,
                             early_stopping=True)

In [ ]:
beam_output

tensor([[   40,   716,   281, 35941,  9345, 23836,    13,   314,   423,   587,
          1762,   319,  9552,   329,   257,   890,   640,   290,   314,   716,
           845,  6568,   546,   262,  2003,   286,  9552,    13,   198,   198,
            40,   423,   257,  1256,   286,  1998,   287,   262,  2214,   286,
         11666,  4430,    13,   554,   262,   938,  1178,   812,    11,   314,
          1053,   587,  2950,   287,   257,  1271,   286,  1180,  4493,    11,
          1390,   262,  2478,   286,   262,  9552,  3859,   329,  3012,   338,
          5565,  3859,    11,   290,   262,  6282,   286,   281,  9552,    12,
         12293,  5175,   598,   329,  4196,   338,  8969,  3859,    13,  2312,
          4493,   423,  2957,   502,   284,   262,  7664,   326,  9552,   318],
        [   40,   716,   281, 35941,  9345, 23836,    13,   314,   423,   587,
          1762,   319,  9552,   329,   257,   890,   640,   290,   314,   716,
           845,  6568,   546,   262,  2003,   286, 

In [ ]:
for i, output in enumerate(beam_output):
    print("{}: {}...".format(i, tokenizer.decode(output,
                                                 skip_special_tokens=True)))
    print('')

0: I am an Artificial Intelligence Developer. I have been working on AI for a long time and I am very excited about the future of AI.

I have a lot of experience in the field of artificial intelligence. In the last few years, I've been involved in a number of different projects, including the development of the AI platform for Google's Android platform, and the creation of an AI-powered mobile app for Apple's iOS platform. These projects have led me to the conclusion that AI is...

1: I am an Artificial Intelligence Developer. I have been working on AI for a long time and I am very excited about the future of AI.

I have a lot of experience in the field of artificial intelligence. In the last few years, I've been involved in a number of different projects, including the development of the AI platform for Google's Android platform, and the creation of an AI-powered mobile app for Apple's iOS platform. These projects have led me to believe that AI is a...

2: I am an Artificial Intellige

In [ ]:
random_output = model.generate(input_ids,
                               do_sample=True,
                               max_length=100,
                               top_k=0,
                               temperature=0.8)
random_output

tensor([[   40,   716,   281, 35941,  9345, 23836,   287, 13643,  2059,    13,
           775,   423,   257,  4950, 42329,  1430,   379,   262,  2059,   286,
         13643,  3706,   284,   905,   703,   284,   787,   257,   983,   546,
         11666,  4430,    13,   314,   481,   467,   284,  4152,   290,  4545,
           379, 13643,   351,   625,  3439,  2444,   290,   625,  2026,  1866,
           286,   674,  1074,    13,   198,   198,  1858,   389,   617,   584,
          3499,  4493,    11,   884,   355, 16828,    12, 25095,  1666,    12,
         10919,    12,  1169,    12,    37, 24905,  1628,   351,   262,  2126,
           286,  4441,   257,   983,   546,  9552,    11,   475,   340,  3073,
           588,   340,   338,  1016,   284,   655,   307,   546,   262,  1830]])

In [ ]:
for i, output in enumerate(random_output):
    print("{}: {}...".format(i, tokenizer.decode(output,
                                                 skip_special_tokens=True)))
    print('')

0: I am an Artificial Intelligence Developer in Oxford University. We have a beautiful internship program at the University of Oxford named to show how to make a game about artificial intelligence. I will go to college and teach at Oxford with over 35 students and over 50 members of our team.

There are some other interesting projects, such as Kid-Knows-what-the-Fifty project with the idea of creating a game about AI, but it looks like it's going to just be about the games...



In [ ]:
top_k_output = model.generate(input_ids,
                              do_sample=True,
                              max_length=100,
                              top_k=50)

for i, output in enumerate(top_k_output):
    print("{}: {}...".format(i, tokenizer.decode(output,
                                                 skip_special_tokens=True)))
    print('')

0: I am an Artificial Intelligence Developer, and I'm very pleased to call you as a client.

As a client, I'm happy to answer questions regarding your program.

Many thanks to all new visitors to my website, such as your question. I would like to add that I have an open process for the developers of this website.

I do not support advertisements, advertisements, or advertisements as such. For example, I think it is important to you to help me do what...

